<a href="https://colab.research.google.com/github/PeterRutkowski/dav-homework1/blob/master/merge_mappers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA

!pip install giotto-tda
from gtda.mapper import *
import pickle

In [0]:
class MyPCA(sklearn.decomposition.PCA):
    def fit_transform(self, X):
        return super().transform(X)
    def fit_transform(self, X, y=None):
        return super().transform(X)

In [0]:
class LatentRep():
    #assumes that all proj in self.projs have .transform() method
    def __init__(self, projectors, label):
        self.projectors = projectors
        self.label = label
    def transform(self, X):
        rep = []
        for proj in self.projectors:
            rep.append( proj.transform(X) )
            #merge all projectors into one vector
            return np.hstack(rep)

In [0]:
def prepare_labels(labels):
  label_mapping = np.load('drive/My Drive/tiny-imagenet-200/data/label_mapping.npy')
  unique_labels = label_mapping[:,0].tolist()

  label_template = []
  for i in range(len(labels)):
    label_index = unique_labels.index(labels[i])
    label = label_mapping[label_index][1]
    label_template.append([i, label, label])

  label_template = np.float64(label_template)
  label_template = np.asarray(label_template)
  return label_template

In [0]:
def import_mapper(cluster_index, n_components=15):
  label = 'PCA_ncomp%d_clus%d' % (n_components, cluster_index)

  frin = open('drive/My Drive/tiny-imagenet-200/PCA_15/%s_latent_rep' % (label), 'rb')
  rep = pickle.load(frin)

  fgin = open('drive/My Drive/tiny-imagenet-200/PCA_15/%s_firstsimplegap_graphs' % (label), 'rb')
  graphs = pickle.load(fgin)

  fpin = open('drive/My Drive/tiny-imagenet-200/PCA_15/%s_mapper_pipes' % (label), 'rb')
  mapper_pipes = pickle.load(fpin)

  data = np.load('drive/My Drive/tiny-imagenet-200/data/train_cluster_%d.npy' % (cluster_index))
  labels = np.load('drive/My Drive/tiny-imagenet-200/data/train_labels_%d.npy' % (cluster_index))

  labels = prepare_labels(labels)

  return np.float64(data), labels, rep, graphs, mapper_pipes

In [0]:
#an algorithm for binarizing training points using
#the computed graph representation
#computes and outputs a CSV file with all training
#points representations , used for training a 
#NeuralNet classifier in the next step.
def binarize_training_points(labels, graphs, mapper_index, cluster_index, n_components=15):
  total_graphbin = []
  total_graphbin.append(labels)

  data_len = 25000
  featlen = 0
  for comp in range(n_components):
    graph = graphs[comp]
    nrnodes = len(graph['node_metadata']['node_id'])
    graphbin = np.zeros((data_len, nrnodes), dtype=int)
    for node in range(nrnodes):
      for pt in graph['node_metadata']['node_elements'][node]:
        graphbin[pt][node] = 1
        
    stamp = str(comp+1)
    total_graphbin.append(graphbin)
    featlen += graphbin.shape[1]
      
  total_graphbinm = np.hstack(total_graphbin)
  print(total_graphbinm.shape)
  label = "mapper%d_cluster%d" % (mapper_index, cluster_index)
  np.save('drive/My Drive/tiny-imagenet-200/PCA_15/matrix_train_%s.npy' % (label), total_graphbinm)
  print('feature vec length=%d' % featlen)

        
  return featlen

In [7]:
x_train_1, labels_1, latent_rep_1, graphs_1, mapper_pipes_1 = import_mapper(1)
featlen_1 = binarize_training_points(labels_1, graphs_1, mapper_index=1, cluster_index=1)

(25000, 5305)
feature vec length=5302


In [8]:
print(type(x_train_1[0][6]))
print(type(labels_1[5][1]))

<class 'numpy.float64'>


In [10]:
x_train_2, labels_2, latent_rep_2, graphs_2, mapper_pipes_2 = import_mapper(2)
featlen_2 = binarize_training_points(labels_2, graphs_2, mapper_index=2, cluster_index=2)

(25000, 5309)
feature vec length=5306


In [11]:
x_train_3, labels_3, latent_rep_3, graphs_3, mapper_pipes_3 = import_mapper(3)
featlen_3 = binarize_training_points(labels_3, graphs_3, mapper_index=3, cluster_index=3)

(25000, 5309)
feature vec length=5306


In [12]:
x_train_4, labels_4, latent_rep_4, graphs_4, mapper_pipes_4 = import_mapper(4)
featlen_4 = binarize_training_points(labels_4, graphs_4, mapper_index=4, cluster_index=4)

(25000, 5305)
feature vec length=5302


In [0]:
# NOT USED NOW
# compute ranges of features in feature vectors
# corresponding to particular filters
def feature_ranges(graphs, n_components=15):
  featureranges = []
  cursum = 0

  for i in range(n_components):
      graph = graphs[i]
      curlen = len( graph['node_metadata']['node_id'] )
      print('component %d, len=%d' % (i, curlen) )
      featureranges.append( (cursum, cursum + curlen - 1) )
      cursum += curlen    
  print(featureranges)

TESTING

---



In [14]:
x_train_1, labels_1, latent_rep_1, graphs_1, mapper_pipes_1 = import_mapper(1)
x_train_2, labels_2, latent_rep_2, graphs_2, mapper_pipes_2 = import_mapper(2)
x_train_3, labels_3, latent_rep_3, graphs_3, mapper_pipes_3 = import_mapper(3)
x_train_4, labels_4, latent_rep_4, graphs_4, mapper_pipes_4 = import_mapper(4)
print(x_train_1.shape, x_train_2.shape, x_train_3.shape, x_train_4.shape)
print(labels_1.shape, labels_2.shape, labels_3.shape, labels_4.shape)

(25000, 12288) (25000, 12288) (25000, 12288) (25000, 12288)
(25000, 3) (25000, 3) (25000, 3) (25000, 3)


In [0]:
def find_alphas(val, intervals, delta = 0.1):
  alphas = []
  if val < intervals[0][1]:
    return tuple([0])
  if val > intervals[-1][0]:
    return tuple([len(intervals)-1])
  for n, intv in enumerate(intervals):
    midv = (intv[0] + intv[1]) / 2
    if abs(val - midv) < abs(intv[1] - intv[0])*(1 + delta) / 2:
      alphas.append(n)
  return tuple(alphas)

ISSUES BELOW

In [0]:
mapper_index = 1
test_index = 1
featlen_mapper = featlen_1

In [24]:
%%time 

# here too much data is imported - purely for convenience
# will be optimised later
x_mapper, labels_mapper, latent_rep_mapper, graphs_mapper, mapper_pipes_mapper = import_mapper(mapper_index)
x_test, labels_test, latent_rep_test, graphs_test, mapper_pipes_test = import_mapper(test_index)

# project the test data
NRNN = 3
latent_data_test = latent_rep_mapper.transform(x_test)

#for each test data, translate its latent representation
#into interval nrs
intervals = []
for mapper_pipe in mapper_pipes_mapper:
    intervals.append(mapper_pipe[1].get_mapper_params()['cover'].get_fitted_intervals())

fullalphas = []
for latent_testp in latent_data_test:
    alphas = []    
    for n, latentv in enumerate(latent_testp):        
        alphas.append(find_alphas(latentv, intervals[n]))
    fullalphas.append(alphas)


[0.]
CPU times: user 20.6 s, sys: 5.81 s, total: 26.4 s
Wall time: 25.3 s


In [57]:
print(len(intervals))
print(intervals[0])
print(len(fullalphas))
print(fullalphas[0])

15
[(-inf, -6840.515541103925), (-7795.531011352855, -4901.54473787125), (-5856.56020812018, -2962.5739346385744), (-3917.5894048875043, -1023.6031314058991), (-1978.618601654829, 915.3676718267761), (-39.647798422154665, 2854.3384750594505), (1899.3230048105215, 4793.309278292127), (3838.2938080431977, 6732.280081524803), (5777.264611275872, 8671.250884757477), (7716.2354145085465, inf)]
25000
[(2, 3), (3, 4), (7, 8), (6, 7), (3, 4), (3, 4), (6, 7), (2, 3), (3, 4), (4,), (6,), (4, 5), (1, 2), (4,), (4, 5)]


In [44]:
%%time
  
#seach for NNeighbour in the preimage of intervals
#precompute NNeighbour for preimage of each interval
int_preim = []
int_nn = {}

#precompute dictionary of fitted NNeighbour for further use
#currently use only single component
#for n in range(len(intervals)):    
n = 0
int_preim_int = []
print('n_intervals', len(intervals[n]))
print(intervals[n])
for i in range(len(intervals[n])):
    nodeids = graphs_mapper[n]['node_metadata']['node_elements'][i] 
    print('nodeids:', len(nodeids))       
    knn = NearestNeighbors(n_neighbors = NRNN, metric='euclidean')
    knn.fit(x_mapper[nodeids])
    int_preim_int.append(nodeids)
    print(len(int_preim_int))
    #knn is fitted knn using x_mapper[nodeids] subset of x_mapper 
    #nodeids are also saved to further reference them in the
    #whole data
    int_nn.update( {(n, tuple([i])): [knn, x_mapper[nodeids], np.array(nodeids)]} )
    if(i > 0):
        knn = NearestNeighbors(n_neighbors = NRNN, metric='euclidean')
        union = list(set(nodeids) | set(int_preim_int[i-1]))
        knn.fit(x_mapper[union])
        int_nn.update( {(n, tuple([i-1,i])): [knn, x_mapper[union], np.array(union)]} )

n_intervals 10
[(-inf, -6840.515541103925), (-7795.531011352855, -4901.54473787125), (-5856.56020812018, -2962.5739346385744), (-3917.5894048875043, -1023.6031314058991), (-1978.618601654829, 915.3676718267761), (-39.647798422154665, 2854.3384750594505), (1899.3230048105215, 4793.309278292127), (3838.2938080431977, 6732.280081524803), (5777.264611275872, 8671.250884757477), (7716.2354145085465, inf)]
nodeids: 125
1
nodeids: 125
2
nodeids: 125
3
nodeids: 125
4
nodeids: 125
5
nodeids: 125
6
nodeids: 125
7
nodeids: 125
8
nodeids: 125
9
nodeids: 125
10
CPU times: user 2.33 s, sys: 30.9 ms, total: 2.37 s
Wall time: 2.35 s


In [47]:
%%time
#for each testpoint compute its representation by finding its NNeighbor
nnids = []
nndists = []
for i in range(len(fullalphas)):    
    nncompids = []
    nncompdists = []
    #for j in range(n_components):        
    #compute this for only the first component (computing for all components is prohibitive)
    j = 0 #take the first component
    #compute knn with distances for each test point (search in the preimage of j-th component)        
    inn = int_nn.get((j, fullalphas[i][j]))
    if(len(inn[2]) > NRNN):
        knn = inn[0].kneighbors([x_test[i]])
        ids = inn[2][knn[1]]
        nncompids.append(ids.squeeze())
        nncompdists.append(knn[0].squeeze())
    else:
        ids = inn[2] #all available points
        dists = np.linalg.norm(x_test[i]-inn[1], axis=1)            
        nncompids.append(ids)
        nncompdists.append(dists)
        
    nnids.append(nncompids) #for each test input , nnids stores a list (per filter function) of 5 NN global ids found in the preimage
    nndists.append(nncompdists) #for each test input , nndists stores a list (per filter function) of 5 NN distances (corresp to the ones in nnids)

CPU times: user 1min 52s, sys: 2.33 ms, total: 1min 52s
Wall time: 1min 52s


In [51]:
print(len(nnids), len(nndists))
print(nnids[0:2])
print(nndists[0:2])

25000 25000
[[array([23037, 18128,  4603])], [array([13310, 22756, 22521])]]
[[array([10732.80867248, 10732.80867248, 10732.80867248])], [array([11704.7759483, 11704.7759483, 11704.7759483])]]


In [56]:
print(np.linalg.norm(x_mapper[0]-x_mapper[4603]))
print(np.linalg.norm(x_mapper[1]-x_mapper[13310]))

10732.808672477116
11704.77594830418


In [16]:
%%time

#generate data matrix with test points representations
#and save to npy file;
#produced by using NNeighbour algorithm

mu = 1e-05
#change names of variables here
neighbor_weights = np.ones_like(nndists)
test_rep = np.zeros((x_test.shape[0], featlen_mapper))

#ASSUMING BELOW, THAT SINGLE COMPONENT HAS BEEN COMPUTED 
#compute the weights for weighting the NN representations
for i in range( len(nndists) ):
    ns = nndists[i][0]
    ns = 1./(ns + mu)
    ns = ns / sum(ns)
    neighbor_weights[i] = ns

total_graphbinm = np.load('drive/My Drive/tiny-imagenet-200/PCA_15/matrix_train_mapper%d_cluster%d.npy' % (mapper_index, mapper_index))
    
#compute the representations using weighted NN
for i  in range(len(nndists)):
    ns = nnids[i]
    features = neighbor_weights[i].reshape(1,NRNN).dot(total_graphbinm[ns, 3:].astype(float))
    test_rep[i] = features

total_test_rep = np.hstack([labels_test, test_rep])
total_test_rep = np.float64(total_test_rep)
print(total_test_rep.shape)
np.save('drive/My Drive/tiny-imagenet-200/PCA_15/matrix_train_mapper%d_cluster%d_2.npy' % (mapper_index, test_index), total_test_rep)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


In [17]:
%%time
test_mapper(1,1, featlen_1)

[0.]
(25000, 5305)
CPU times: user 2min 16s, sys: 9.61 s, total: 2min 25s
Wall time: 2min 52s


In [18]:
test1 = np.load('drive/My Drive/tiny-imagenet-200/PCA_15/matrix_train_mapper1_cluster1.npy')
test2 = np.load('drive/My Drive/tiny-imagenet-200/PCA_15/matrix_train_mapper1_cluster1_2.npy')
print(test1.shape, test2.shape)
print(type(test1[4][4]), type(test2[4][4]))
print(np.unique(test2-test1))

(25000, 5305) (25000, 5305)
<class 'numpy.float64'> <class 'numpy.float64'>
[-1.  0.  1.]


In [19]:
print(np.linalg.norm(test1-test2))

1062.4264680437889


In [20]:
labels_1

array([[0.0000e+00, 1.4100e+02, 1.4100e+02],
       [1.0000e+00, 1.1800e+02, 1.1800e+02],
       [2.0000e+00, 4.7000e+01, 4.7000e+01],
       ...,
       [2.4997e+04, 9.9000e+01, 9.9000e+01],
       [2.4998e+04, 1.3700e+02, 1.3700e+02],
       [2.4999e+04, 5.8000e+01, 5.8000e+01]])